In [1]:
# import necessary libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# BigQuery settings
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Load the experience data

In [2]:
X1 = np.load('features/user_exp_score.npy').reshape((-1,1))
X2 = np.load('features/avg_user_joint_exp.npy').reshape((-1,1))

Y = np.load('features/t2tpa.npy')



(264, 2)


### Load the busyness data

In [9]:
# load the patient enc id:

from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();
query = "select jc_uid, pat_enc_csn_id_coded, t2tpa from `noshad.cohort_v2`"; # Example dataset table
cursor.execute(query);
results = cursor.fetchall();

A = np.array(results)
print(A.shape)
print(A[0,:])
Enc_ID = A[:,1]  


# Create the distionary




Pat_dic = {}

for id in Enc_ID:
    #print(id)
    #print(np.where(A[:,1]==id)[0])
    #print(A[np.where(A[:,1]==id)[0],2])
    t2tpa = int(A[np.where(A[:,1]==id)[0],2])
    pat_id = A[np.where(A[:,1]==id)[0],0]
    
    #print(t2tpa)
    if t2tpa < 60*10:
        Pat_dic[id] = {} #each patient has a dictionary
        Pat_dic[id]['t2tpa'] = t2tpa
        Pat_dic[id]['pat_id'] = pat_id



# Extract all action busyness:

client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();
query = "select jc_uid, pat_enc_csn_id_coded, dep_busy_norm_all_actions from `noshad.Feature_dep_busy_all_actions`"; # Example dataset table
cursor.execute(query);
results = cursor.fetchall();

A = np.array(results)
print(A.shape)
print(A[0,:])


#with open('features/num_tnx.csv', 'r') as f:
#    A_temp = csv.reader(f)
#    temp = list(A_temp)

#print(temp[0]) # The column names

#A = np.array(temp[1:]) # array of t2tpa


for id in Pat_dic.keys():
    if A[np.where(A[:,1]==id)[0],2].shape[0] > 0:
        num_actions = float(A[np.where(A[:,1]==id)[0],2])
        Pat_dic[id]['dep_busy_all_actions'] = num_actions
    else:
        print(id)
        Pat_dic[id]['dep_busy_all_actions'] = 0


        
# Busyness to numpy
X3 = np.array([Pat_dic[id]['dep_busy_all_actions'] for id in Pat_dic.keys()] )

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


(264, 3)
['JCcd691a' '131018562095' '39']


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


(260, 3)
['JCcb69cc' '131029086038' '125.33333333333333']
131042907349
131014276335
131014194983
131009784135


### Clean the data and split to train, test

In [11]:
# Replace Nan with -1:
X1[np.isnan(X1[:,0])]= -1
X2[np.isnan(X1[:,0])]= -1
X3[np.isnan(X1[:,0])]= -1

Y[np.isnan(X1[:,0])]= -1

X = np.concatenate((X1, X2, X3.reshape((-1,1))), axis=1)
N = X.shape[0]
print(X.shape)

# train and test

N_train = 200

X_train = X[:N_train,:]
X_test = X[N_train:,:]

Y_train = Y[:N_train]
Y_test = Y[N_train:]

(264, 3)


### Prediction Model

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as MSE

# Fit regression model
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_poly2 = SVR(kernel='poly', degree=2)#, C=100, gamma='auto', degree=4)
svr_poly3 = SVR(kernel='poly', degree=3)#, C=100, gamma='auto', degree=4)
svr_poly4 = SVR(kernel='poly', degree=4)#, C=100, gamma='auto', degree=4)
svr_poly5 = SVR(kernel='poly', degree=5)#, C=100, gamma='auto', degree=4)


Y_pred = svr_poly.fit(X_train, Y_train).predict(X_test)
print(np.mean(Y_pred))


print('Y statistics',np.mean(Y), np.std(Y))
#clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(svr_rbf, X, Y, cv=5, scoring='neg_mean_absolute_error') #scoring = 'neg_mean_absolute_error')
print('RBF: ',-scores, np.mean(-scores))

scores = cross_val_score(svr_lin, X, Y, cv=5, scoring='neg_mean_absolute_error') #scoring = 'neg_mean_absolute_error')
print('Linear: ',-scores, np.mean(-scores))

scores = cross_val_score(svr_poly2, X, Y, cv=5, scoring='neg_mean_absolute_error') #scoring = 'neg_mean_absolute_error')
print('poly2: ',-scores, np.mean(-scores))

scores = cross_val_score(svr_poly3, X, Y, cv=5, scoring='neg_mean_absolute_error') #scoring = 'neg_mean_absolute_error')
print('poly3: ',-scores, np.mean(-scores))
scores = cross_val_score(svr_poly4, X, Y, cv=5, scoring='neg_mean_absolute_error') #scoring = 'neg_mean_absolute_error')
print('poly4: ',-scores, np.mean(-scores))
scores = cross_val_score(svr_poly5, X, Y, cv=5, scoring='neg_mean_absolute_error') #scoring = 'neg_mean_absolute_error')
print('poly5: ',-scores, np.mean(-scores))

50.18000572961695
Y statistics 56.208333333333336 38.6079528266476
RBF:  [26.55389636 33.00544152 26.10358167 26.95047568 28.39020259] 28.20071956334372
Linear:  [25.4184934  29.47047541 18.70258583 25.37941711 21.79451928] 24.153098206079868
poly2:  [25.15527264 30.57775084 21.06519911 27.99958752 23.92649424] 25.74486086704557
poly3:  [25.06532244 30.5149656  21.22246579 28.44417377 23.90586541] 25.83055860086778
poly4:  [25.02566095 30.46416623 21.28476309 28.15490504 23.82040707] 25.749980476173857
poly5:  [25.00433956 30.09979739 21.24549392 29.76741303 23.73030411] 25.96946960198274


### Multivariate Correlation Analysis

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as MSE
from scipy.stats.stats import pearsonr 


# Fit regression model
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_poly2 = SVR(kernel='poly', degree=2)#, C=100, gamma='auto', degree=4)
svr_poly3 = SVR(kernel='poly', degree=3)#, C=100, gamma='auto', degree=4)
svr_poly4 = SVR(kernel='poly', degree=4)#, C=100, gamma='auto', degree=4)
svr_poly5 = SVR(kernel='poly', degree=5)#, C=100, gamma='auto', degree=4)


Y_pred = svr_lin.fit(X_train, Y_train).predict(X_test)
#print(np.mean(Y_pred))
print(pearsonr(np.reshape(Y_pred,(-1,)), np.reshape((Y_test),(-1,))))

Y_pred = svr_poly2.fit(X_train, Y_train).predict(X_test)
#print(np.mean(Y_pred))
print(pearsonr(np.reshape(Y_pred,(-1,)), np.reshape((Y_test),(-1,))))

Y_pred = svr_poly3.fit(X_train, Y_train).predict(X_test)
#print(np.mean(Y_pred))
print(pearsonr(np.reshape(Y_pred,(-1,)), np.reshape((Y_test),(-1,))))

Y_pred = svr_poly4.fit(X_train, Y_train).predict(X_test)
#print(np.mean(Y_pred))
print(pearsonr(np.reshape(Y_pred,(-1,)), np.reshape((Y_test),(-1,))))

Y_pred = svr_poly5.fit(X_train, Y_train).predict(X_test)
#print(np.mean(Y_pred))
print(pearsonr(np.reshape(Y_pred,(-1,)), np.reshape((Y_test),(-1,))))

Y_pred = svr_rbf.fit(X_train, Y_train).predict(X_test)
#print(np.mean(Y_pred))
print(pearsonr(np.reshape(Y_pred,(-1,)), np.reshape((Y_test),(-1,))))



(0.4446228363625163, 0.0002324858484278026)
(0.3147235471223256, 0.01131274097923752)
(0.2924594153637401, 0.019022603410975312)
(0.3366525101612707, 0.006527626011233836)
(0.31780921200594053, 0.010494807795011397)
(0.093667571180747, 0.4616135728404382)


In [ ]:

Y_pred = svr_rbf.fit(X_train, Y_train).predict(X_test)
print(np.mean(Y_pred))

r = svr_rbf.fit(X_train, Y_train).score(X_test,Y_test)
print(r)
print(svr_rbf.fit(X_train, Y_train).predict(X_test)[:20])
print(Y_test[:20])
print('MSE',np.sqrt(MSE(Y_test, Y_pred)))


print(scores)
print(Y[:20])

